In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_154336f42d194f25bb030b76e7f109fd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZY5bk4X9TizH7sKrJ7yNyKvSv9iWIV7ei78dWwXuCj9S',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_154336f42d194f25bb030b76e7f109fd.get_object(Bucket='sentimentanalysisofcovid19tweets-donotdelete-pr-x01mtp2wbei3mi',Key='Ready corona tweets (2).csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

tweets= pd.read_csv(body)
tweets.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,id_str,tweet,truncated:false,senti_score,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144
0,id:1240727821028405200,"id_str:""1240727821028405249""","full_text:""RT @ebs_the_great: A lot of you wou...",truncated:false,0.357143,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id:1240727820629946400,"id_str:""1240727820629946368""","full_text:""RT @MakedaMorrison: Day 5: We have ...",truncated:false,0.116071,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id:1240727810249044000,"id_str:""1240727810249043969""","full_text:""RT @Damii_aros: Mayorkun is somewhe...","this your body dey killing person like Corona\""""",-0.050000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id:1240727809300902000,"id_str:""1240727809300901888""","full_text:""RT @SAIBdelvi: I appreciate both Pa...",truncated:false,-0.714286,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id:1240727817555447800,"id_str:""1240727817555447808""","full_text:""RT @iam_erhnehst: Everything was fi...",truncated:false,0.700000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
index = tweets.index
columns = tweets.columns
values = tweets.values
tweets_new=tweets[['tweet','senti_score']]
tweets_new.head()                   

,tweet,senti_score
0,"full_text:""RT @ebs_the_great: A lot of you wou...",0.357143
1,"full_text:""RT @MakedaMorrison: Day 5: We have ...",0.116071
2,"full_text:""RT @Damii_aros: Mayorkun is somewhe...",-0.050000
3,"full_text:""RT @SAIBdelvi: I appreciate both Pa...",-0.714286
4,"full_text:""RT @iam_erhnehst: Everything was fi...",0.700000


In [4]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def preprocess(tweet):
    tweet.lower()
    tweet = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', tweet)
    tweet = re.sub("(@[A-Za-z0-9_]+)","", tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
   # emoji_pattern = re.compile("["
          #                 u"\U0001F600-\U0001F64F"  # emoticons
          #                 u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          #                 u"\U0001F680-\U0001F6FF"  # transport & map symbols
          #                 u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          #                 u"\U00002702-\U000027B0"
          #                 u"\U000024C2-\U0001F251"
          #                 "]+", flags=re.UNICODE)
   # tweet=emoji_pattern.sub(r'', tweet)
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    return " ".join(filtered_words)
    #lemmatizer = WordNetLemmatizer()
    #lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    


tweets_new['tweet']= tweets_new['tweet'].apply(preprocess)    
#print(tweets_new['tweet'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
tweets_new



,tweet,senti_score
0,fulltextRT A lot would actually benefit taking...,0.357143
1,fulltextRT Day 5 We rediscovered farming,0.116071
2,fulltextRT Mayorkun somewhere studio singing R...,-0.050000
3,fulltextRT I appreciate Pak Jokowi Pak Prabowo...,-0.714286
4,fulltextRT Everything fine world Liverpool dec...,0.700000
5,fulltextRT ’ deleted scene TWD ’ special coron...,0.000000
6,fulltextRT GoodnnIt came China,0.000000
7,fulltextRT My dick Bigger Corona virusnMy dick...,0.216667
8,fulltextRT The man save us corona virus,-0.200000
9,fulltextAha Knew Another hidden family issue e...,0.000000


In [6]:
def result(score):
    if score < 0:
        return 'negative'
    elif score == 0:
        return 'neutral'
    else:
        return 'positive'
    
tweets_new['Result']=tweets_new['senti_score'].apply(result)    
tweets_new

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,tweet,senti_score,Result
0,fulltextRT A lot would actually benefit taking...,0.357143,positive
1,fulltextRT Day 5 We rediscovered farming,0.116071,positive
2,fulltextRT Mayorkun somewhere studio singing R...,-0.050000,negative
3,fulltextRT I appreciate Pak Jokowi Pak Prabowo...,-0.714286,negative
4,fulltextRT Everything fine world Liverpool dec...,0.700000,positive
5,fulltextRT ’ deleted scene TWD ’ special coron...,0.000000,neutral
6,fulltextRT GoodnnIt came China,0.000000,neutral
7,fulltextRT My dick Bigger Corona virusnMy dick...,0.216667,positive
8,fulltextRT The man save us corona virus,-0.200000,negative
9,fulltextAha Knew Another hidden family issue e...,0.000000,neutral


In [23]:

vectorizer = TfidfVectorizer(sublinear_tf=True)
#y = tweets_new.senti_score  # dependent variable
x = vectorizer.fit_transform(tweets_new.tweet)
#x=vectorizer.fit_transform(tweets_new.tweet)
y= tweets_new.Result
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)
NB_model = MultinomialNB()
NB_model.fit(x_train, y_train)
y_predict= NB_model.predict(x_test)


In [13]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient 

In [14]:
wml_credentials={
  "apikey": "-uYDMhtTvj2kAcF5jI4iNgtJSvhcfyGsmP3u-Z8Cbhz-",
  "iam_apikey_description": "Auto-generated for key e2cd5064-3e51-4818-8488-23d1d6ecbdcc",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/bd58a9e140744e90a94d9a932148f797::serviceid:ServiceId-e766f9dc-6621-4d2e-8452-9d9866ee4b7c",
  "instance_id": "825b630a-7afc-41aa-879c-f04604b7fc19",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [15]:
client=WatsonMachineLearningAPIClient(wml_credentials)

In [16]:
metadata={
    client.repository.ModelMetaNames.AUTHOR_NAME :"The Py-Squad",
    client.repository.ModelMetaNames.AUTHOR_EMAIL:"prab17127.ei@rmkec.ac.in",
    client.repository.ModelMetaNames.NAME:"SentimentAnalysisOfCovid-19Tweets"
    
}

In [17]:
stored_data=client.repository.store_model(NB_model,meta_props=metadata)

In [18]:
guid=client.repository.get_model_uid(stored_data)
guid


'45cd16ea-f993-4868-80ed-66d4c1c85a29'

In [19]:
deploy=client.deployments.create(guid)



#######################################################################################

Synchronous deployment creation for uid: '45cd16ea-f993-4868-80ed-66d4c1c85a29' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='57cbcd95-bf5c-48ea-8988-cde23c22d1a0'
------------------------------------------------------------------------------------------------




In [20]:
scoring_end_point=client.deployments.get_scoring_url(deploy)
scoring_end_point


'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/825b630a-7afc-41aa-879c-f04604b7fc19/deployments/57cbcd95-bf5c-48ea-8988-cde23c22d1a0/online'